In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import html5lib

In [2]:
def makeSoup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

The code below is a way to try and find the special characteristics of the episode. Check back later on how this works.

In [3]:
def findBetween( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def findBetweenR( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [4]:
def removePrefix(text, prefix):
    return text[len(prefix):] if text.startswith(prefix) else text
        

In [5]:
soup = makeSoup('https://en.wikipedia.org/wiki/The_Bachelorette_(season_4)')

In [112]:
def soupCleanser(soup):
    import re
    import numpy as np
    import pandas as pd
    # make the soup
    
    # find the right table
    tables = soup.findChildren('table')
    table = tables[2]
    
    # find the descriptive colors in hex format
    descript_colors_cols = soup.findChildren('dd')
    styles = [color.span['style'] for color in descript_colors_cols]
    colors = [findBetween(color,"background-color:",";") for color in styles]
    
    
    messy_color_codes = [dd.text for dd in descript_colors_cols]
    color_codes = [removePrefix(code,"\xa0\xa0\xa0\xa0 ") for code in messy_color_codes]
    
    color_dict = dict(zip(colors, color_codes))

    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    body = data[1:][1:]
    cols   =  zip(*body)
    
    body = [[re.sub(r'\[\w+\]', '', i) for i in row]
        for row in body]
    
    for i in body:
        if len(i) == len(body[0]):
            pass
        else:
            a = [" "]
            a.extend(" " * (len(body[0])))
            diff = len(a) - len(i) 
            n = i.extend( a[0:diff])
                
    contestants = [row[0] for row in body]
    
    # create lists of call out
    call_out_order_list = []
    for i in list(np.arange(0,(len(body[0])-1))):
        c = [row[i] for row in body] # column
        ci = list(np.arange(1,len(c) + 1)) # column index/counter
        cdict = dict(zip(c, ci)) # creaet dictionary
        col_out_col = [cdict.get(k, 'eliminated') for k in contestants] # call out order
        call_out_order_list.append(col_out_col)
    
    call_out_order_list[0] = contestants # replace first row with contestant names
    
    # find headers for data frame creation
    d = [[td.text for td in row.select('th')]
        for row in table.findAll('tr')]
    
    season_headers = d[1]
    header = ["contestants"] + season_headers  
    
    vov = dict(zip(header, call_out_order_list))
    
    return pd.DataFrame(vov)

In [113]:
soupCleanser(soup)

,1,2,3,4,5,6,7,contestants
0,15,7,7,7,eliminated,eliminated,eliminated,Brian - TX
1,11,1,eliminated,eliminated,eliminated,eliminated,eliminated,Paul
2,5,2,8,6,3,4,eliminated,Graham
3,8,10,9,2,eliminated,eliminated,eliminated,Sean
4,3,11,12,eliminated,eliminated,eliminated,eliminated,Richard
5,14,8,2,3,2,3,2,Jason
6,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,Spero
7,2,5,5,1,4,1,1,Jesse
8,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,John
9,10,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,Chris


In [21]:
import re
#for name in tbl_d['name']:
cleaned_name = re.sub(r'\[\w+\]', '', "james")
#new_names.append(cleaned_name)
cleaned_name

'james'

# old code

In [69]:
#def dataCollector(soup):
#    import re
#    import numpy as np
#    # make the soup
#    this = makeSoup(soup)
#    
#    # find the right table
#    tables = this.findChildren('table')
#    table = tables[2]
#    
#    
#    # find the descriptive colors in hex format
#    descript_colors_cols = this.findChildren('dd')
#    styles = [color.span['style'] for color in descript_colors_cols]
#    colors = [findBetween(color,"background-color:",";") for color in styles]
#    
#    
#    messy_color_codes = [dd.text for dd in descript_colors_cols]
#    color_codes = [removePrefix(code,"\xa0\xa0\xa0\xa0 ") for code in messy_color_codes]
#    
#    color_dict = dict(zip(colors, color_codes))
#
#    # turn the data into a workable format
#    data   = [[td.text for td in row.select('td')]
#             for row in table.findAll('tr')]
#    
#    body = data[1:][1:]
#    cols   =  zip(*body)
#    
#    body_n = []
#
#    for i in body:
#        if len(i) == len(body[0]):
#            body_n.append(i)
#        else:
#            a = [" "]
#            a.extend(" " * (len(body[0])))
#            diff = len(a) - len(i) 
#            n = i.extend( a[0:diff])
#            body_n.append(n)
#                
#    # create a dict with the data
#    #tbl_d  = {name:col for name, col in zip(header,cols)}
#    contestants = [row[0] for row in body]
#    
#    #c2 = [row[2] for row in body]
#    #ci2 = list(np.arange(1,len(c2) + 1))
#    #c2dict = dict(zip(c2, ci2))
#    
#    #yup = [c2dict[x] for x in contestants]
#    #yup = [c2dict.get(k, 'eliminated') for k in contestants] # default to eliminated if name doesn't exist
#    
#    elimination_list = []
#    for i in list(np.arange(0,(len(body[0])-1))):
#        c = [row[i] for row in body]
#        ci = list(np.arange(1,len(c) + 1))
#        cdict = dict(zip(c, ci))
#        elim_col = [cdict.get(k, 'eliminated') for k in contestants]
#        elimination_list.append(elim_col)
#          
#    return elimination_list

In [70]:
#body = dataCollector('https://en.wikipedia.org/wiki/The_Bachelorette_(season_4)')